In [1]:
import pandas as pd
import os
import plotly.express as px
# Läs in data
path = os.path.join(os.getcwd(),"data","athlete_events.csv")
os_data = pd.read_csv(path)
# Checka grund data
os_data.info()
print(f"Totalt är {os_data['NOC'].nunique()} länder med i datan, de är {os_data['NOC'].unique()}")
print(f"Totalt har {os_data['Sport'].nunique()} sport, de är {os_data['Sport'].unique()}")
print(f"Medaljtyper är {os_data['Medal'].unique()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB
Totalt är 230 länder med i datan, de är ['CHN' 'DEN' 'NED' 'USA' 'FIN' 'NOR' 'ROU' 'EST' 'FRA' 'MAR' 'ESP' 'EGY'
 'IRI' 'BUL' 'ITA' 'CHA' 'AZE' 'SUD' 'RUS' 'ARG' 'CUB' 'BLR' 'GRE'

In [ ]:
#Statistik för ålden
os_data['Age'].describe()


count    261642.000000
mean         25.556898
std           6.393561
min          10.000000
25%          21.000000
50%          24.000000
75%          28.000000
max          97.000000
Name: Age, dtype: float64

In [ ]:
# Checka null värde
nulls = os_data.isnull().sum()
nulls

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

In [3]:
# Diagram över könsfördelningen
os_gender = os_data.groupby(["Year","Sex"]).size().reset_index(name = "Count")
fig = px.bar(os_gender, x = "Year", y="Count",color= "Sex", title="Gender distribution")
fig.show()


In [2]:
# Diagram över topp 10 länder som tagit flest medaljer
os_topp10 = os_data[["Team","Medal"]].groupby("Team")["Medal"].count().sort_values(ascending=False).head(10).reset_index()
os_topp10
px.bar(os_topp10, x="Medal", y="Team",color = "Team",title="Topp 10 country")


In [5]:
#  anonymisera  idrottarnas namn med hashfunktionen SHA-256
import hashlib
os_data["Name_hash"]=os_data["Name"].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
os_hash_name = os_data.drop(columns="Name")
os_hash_name.head()

,ID,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Name_hash
0,1,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,3a4eef48434c66b3f14ab0221f6762d0ef7c6135ab2790...
1,2,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,a6430cc6630934275dc6283f7e97e9625e6587cdddec7a...
2,3,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,9c198b205332c2c8e1542e0f9534b9e270780a41d978ec...
3,4,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,0a477bb1c5ad39716f9c775e54d18d16aa8b37ada55548...
4,5,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,5b7be356aa28178096dc6747f0b8e4e393eaceb5f95310...


In [6]:
# Tvätta data för Östrike 
os_Austria = os_hash_name[os_hash_name["Team"]=="Austria"]
os_Austria.head(2)

,ID,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Name_hash
1738,968,F,16.0,NaN,NaN,Austria,AUT,1912 Summer,1912,Summer,Stockholm,Swimming,Swimming Women's 100 metres Freestyle,NaN,75ced3fe1cd3e805cbabb3a33e16c4c06f52fc0d562f67...
1739,968,F,16.0,NaN,NaN,Austria,AUT,1912 Summer,1912,Summer,Stockholm,Swimming,Swimming Women's 4 x 100 metres Freestyle Relay,Bronze,75ced3fe1cd3e805cbabb3a33e16c4c06f52fc0d562f67...


In [22]:
topp_medal = os_Austria.groupby("Sport")["Medal"].count().sort_values(ascending = False).reset_index().head(10)
fig = px.bar(topp_medal,x="Medal",y="Sport",color="Sport", title = "Topp10 Sport")
fig.show()

In [16]:
year_medal = os_Austria.groupby("Year")["Medal"].count().reset_index()
fig = px.bar(year_medal, x="Year",y="Medal", title = "Numbel of medal per OS")
fig.show()

In [29]:
fig = px.histogram(os_Austria, x = "Age",nbins =20, color = "Sex")
fig.show()